Now we need to perform an inverse text normalization to obtain examples to train an LLM on.

In [ ]:
import json
from tqdm.notebook import tqdm
import random

In [ ]:
from collections import defaultdict, Counter

In [ ]:
PATHES = {
    "librusec": {
        "input": "/home/jovyan/wdc1/datasets/_PLAIN/librusec",
        "output": "/home/jovyan/data/librusec.jsonl",
    },
    "ficbook": {
        "input": "/home/jovyan/wdc1/datasets/_PLAIN/ficbook",
        "output": "/home/jovyan/data/ficbook.jsonl",
    },
    "pikabu": {
        "input": "/home/jovyan/wdc1/datasets/_WEB20/pikabu",
        "output": "/home/jovyan/data/pikabu.json",
    },
}

Uncomment pairs of input and output file pathes below one by one.

In [ ]:
DATASET = "librusec"
DATASET = "ficbook"
DATASET = "pikabu"

In [ ]:
# !pip install jsonlines pysimdjson

In [ ]:
import simdjson


data = []
parser = simdjson.Parser()
with open(PATHES[DATASET]["input"]) as f:
    for i, line in tqdm(enumerate(f)):
        data.append(parser.parse(line).as_dict())
random.choice(data)

In [ ]:
for elem in tqdm(data):
    for text_elem in elem["texts"]:
        text_elem["matches"] = [_["form"] for _ in text_elem["matches"]]
random.choice(data)

~~I end up using `nemo_text_processing` as it is fast.~~
~~This part is vital so better use another one here next time.~~

~~Seems ok for a first attempt anyway.~~

In [ ]:
# from nemo_text_processing.inverse_text_normalization.inverse_normalize import InverseNormalizer
# inverse_normalizer = InverseNormalizer(lang='ru')

Use [another](https://github.com/flockentanz/word_to_number_ru) itn now as NeMo covers not much of what I want to.

Do `pip install natasha==0.10.0 yargy==0.12.0` first.

In [ ]:
from word_to_number_ru.extractor import NumberExtractor


ne = NumberExtractor()

In [ ]:
sum(len(_["texts"]) for _ in data)

Make some **ROUGH** balancing here.
First, perform a quick distribution check.

In [ ]:
counter = Counter()
for elem in tqdm(data):
    for text in elem["texts"]:
        counter.update(text["matches"])
sum(counter.values()), len(counter), counter

Seems too much of numbers less than 10.
So we strip a part of the most common numbers.

In [ ]:
from multiprocessing import Process, Queue
from multiprocessing import Pool
from itertools import islice, chain


queue = Queue()
occurances = Counter()
occurances_limit = sum(counter.values()) / 25  # len(counter)
print(f"{occurances_limit=}")


def inverse_normalize(elem):
    result = []
    for i_text, text in enumerate(elem["texts"]):
        matches = [match for match in text["matches"] if occurances[match] < occurances_limit]
        if not matches:
            continue
        occurances.update(text["matches"])
        # itn = inverse_normalizer.inverse_normalize(text["text"], verbose=False)
        itn = ne(text["text"])
        if itn == text["text"]:
            continue
        result.append({
            "tn": text["text"],
            "itn": itn,
            "orig_index": elem["index"],
            "text_index": i_text
        })
    return result


b_size = 10000  # you may wnat to decrease it in case of librusec as its texts are vast
f = open(PATHES[DATASET]["output"], "w")
i = 0
pbar = tqdm(total=len(data))
while i < len(data):
    with Pool(15) as p:
        result = p.imap_unordered(inverse_normalize, data[i:i + b_size])
        for elem in chain(*result):
            json.dump(elem, f, ensure_ascii=False)
            f.write("\n")
        i += b_size
        pbar.update(b_size)
pbar.close()
f.close()

In [ ]:
occurances